Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [1]:
import Pkg #this line doesn't seem to run either way for me
Pkg.add("Pkg")
Pkg.add("Dates")

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [3]:
"""This activates packages, namely ambulance deployment"""

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON, DelimitedFiles
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

using AmbulanceDeployment, Dates

 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`


In [4]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 1 method)

In [5]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
#include("..//src//evaluate.jl") #if you include, you need to include JuMP
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 40 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [19]:
x = ones(1,44)
x[2:5] = x[2:5] .- 1
x

1×44 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [6]:
"""Simulation Enginer"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [20]:
"""call simulation"""
#@time df, guiArray = simulate_events!(problem, dispatch)
df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
Ambulance has arrived on the scene for event 1 at time 338
calling event id: 2 time: 575 value: 38
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
Ambulance for event 1 has arrived at the hospital at time 1021
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
Ambulance has arrived on the scene for event 2 at time 1250
Ambulance has arrived on the scene for event 4 at time 1302
Ambulance has arrived on the scene for event 3 at time 1407
calling event id: 7 time: 1450 value: 76
Ambulance has arrived on the scene for event 6 at time 1487
calling event id: 8 time: 1578 value: 67
calling event id: 9 time: 1640 value: 52
Ambulance has arrived on the scene for event 5 at time 1752
calling event id: 10 time: 1838 value: 67
Ambulance has arrived on the scene for event 8 at time 1878
calling event id: 11 time: 1899 value: 67
calling event id: 12 time: 1901 value: 82
Ambulance for event 4 has

calling event id: 37 time: 10091 value: 98
Ambulance has arrived on the scene for event 36 at time 10152
Ambulance for event 29 is returning to the station at 10271
Ambulance for event 27 is returning to the station at 10463
Ambulance for event 32 has arrived at the hospital at time 10550
calling event id: 38 time: 10595 value: 98
Ambulance for event 34 has arrived at the hospital at time 10640
Ambulance from event 27 has returned to station 17
Ambulance from event 29 has returned to station 3
Ambulance for event 35 has arrived at the hospital at time 10689
Ambulance from event 21 has returned to station 31
Ambulance has arrived on the scene for event 37 at time 10699
calling event id: 39 time: 10727 value: 52
calling event id: 40 time: 10770 value: 98
Ambulance for event 30 is returning to the station at 10803
Ambulance for event 31 is returning to the station at 10832
Ambulance for event 28 is returning to the station at 11041
Ambulance for event 36 has arrived at the hospital at tim

Ambulance from event 55 has returned to station 17
Ambulance for event 60 is returning to the station at 19532
calling event id: 64 time: 19691 value: 51
Ambulance has arrived on the scene for event 63 at time 19879
calling event id: 65 time: 19963 value: 98
Ambulance has arrived on the scene for event 64 at time 19991
Ambulance for event 62 has arrived at the hospital at time 20013
Ambulance for event 59 is returning to the station at 20115
Ambulance from event 60 has returned to station 1
Ambulance for event 61 has arrived at the hospital at time 20384
calling event id: 66 time: 20471 value: 113
Ambulance from event 59 has returned to station 14
Ambulance has arrived on the scene for event 65 at time 20785
calling event id: 67 time: 20814 value: 36
Ambulance for event 64 has arrived at the hospital at time 20977
calling event id: 68 time: 21018 value: 66
Ambulance for event 63 has arrived at the hospital at time 21055
Ambulance has arrived on the scene for event 66 at time 21057
Ambu

Ambulance for event 92 is returning to the station at 32203
Ambulance from event 87 has returned to station 22
calling event id: 93 time: 32356 value: 66
Ambulance from event 89 has returned to station 25
Ambulance for event 91 is returning to the station at 32568
Ambulance has arrived on the scene for event 93 at time 32656
Ambulance from event 91 has returned to station 8
Ambulance from event 92 has returned to station 14
calling event id: 94 time: 32957 value: 66
Ambulance for event 90 is returning to the station at 33225
Ambulance has arrived on the scene for event 94 at time 33257
Ambulance from event 90 has returned to station 39
Ambulance for event 93 has arrived at the hospital at time 33564
calling event id: 95 time: 33904 value: 99
calling event id: 96 time: 33986 value: 142
Ambulance has arrived on the scene for event 95 at time 34204
calling event id: 97 time: 34426 value: 66
Ambulance has arrived on the scene for event 96 at time 34509
Ambulance for event 94 has arrived at

calling event id: 127 time: 45780 value: 97
Ambulance has arrived on the scene for event 124 at time 45854
Ambulance has arrived on the scene for event 126 at time 45976
Ambulance for event 122 has arrived at the hospital at time 46026
Ambulance for event 120 has arrived at the hospital at time 46050
Ambulance for event 115 is returning to the station at 46072
calling event id: 128 time: 46110 value: 82
Ambulance has arrived on the scene for event 127 at time 46154
Ambulance from event 115 has returned to station 8
Ambulance has arrived on the scene for event 125 at time 46289
Ambulance for event 121 has arrived at the hospital at time 46378
calling event id: 129 time: 46504 value: 141
calling event id: 130 time: 46522 value: 68
Ambulance for event 123 has arrived at the hospital at time 46637
Ambulance for event 124 has arrived at the hospital at time 46743
Ambulance for event 116 is returning to the station at 46776
Ambulance has arrived on the scene for event 128 at time 46778
Ambul

...

Ambulance from event 912 has returned to station 30
calling event id: 922 time: 259059 value: 69
922: call from 69 is a shortfall
Ambulance from event 910 has returned to station 2
calling event id: 922 time: 259063 value: 69
922: call from 69 is a shortfall
Ambulance from event 914 has returned to station 11
calling event id: 922 time: 259111 value: 69
This is a shortfallcalling event id: 926 time: 259164 value: 66
calling event id: 927 time: 259209 value: 51
Ambulance for event 901 is returning to the station at 259283
Ambulance for event 919 has arrived at the hospital at time 259300
Ambulance for event 917 is returning to the station at 259357
Ambulance from event 913 has returned to station 15
Ambulance has arrived on the scene for event 924 at time 259391
calling event id: 928 time: 259399 value: 82
Ambulance has arrived on the scene for event 926 at time 259464
Ambulance from event 895 has returned to station 44
Ambulance has arrived on the scene for event 923 at time 259492
Amb

952: call from 128 is a shortfall
Ambulance for event 938 is returning to the station at 263879
Ambulance for event 941 has arrived at the hospital at time 263921
Ambulance for event 939 is returning to the station at 263927
Ambulance has arrived on the scene for event 937 at time 263946
Ambulance for event 940 is returning to the station at 263949
Ambulance has arrived on the scene for event 950 at time 264008
Ambulance for event 945 has arrived at the hospital at time 264015
calling event id: 953 time: 264027 value: 98
Ambulance from event 938 has returned to station 8
calling event id: 952 time: 264031 value: 128
952: call from 128 is a shortfall
calling event id: 954 time: 264093 value: 141
Ambulance for event 946 has arrived at the hospital at time 264125
Ambulance for event 933 is returning to the station at 264158
Ambulance for event 934 is returning to the station at 264344
Ambulance from event 940 has returned to station 3
calling event id: 952 time: 264360 value: 128
952: cal

calling event id: 973 time: 269600 value: 127
Ambulance for event 967 has arrived at the hospital at time 269688
Ambulance has arrived on the scene for event 972 at time 269689
Ambulance for event 964 is returning to the station at 269715
Ambulance for event 966 is returning to the station at 269723
Ambulance has arrived on the scene for event 973 at time 269900
Ambulance for event 965 is returning to the station at 269913
Ambulance has arrived on the scene for event 970 at time 270005
Ambulance has arrived on the scene for event 971 at time 270065
Ambulance from event 958 has returned to station 2
Ambulance from event 957 has returned to station 43
Ambulance from event 966 has returned to station 26
Ambulance for event 972 has arrived at the hospital at time 270385
Ambulance for event 969 has arrived at the hospital at time 270410
Ambulance for event 973 has arrived at the hospital at time 270454
calling event id: 974 time: 270537 value: 98
Ambulance from event 964 has returned to sta

calling event id: 998 time: 282347 value: 116
Ambulance for event 995 is returning to the station at 282515
Ambulance for event 996 has arrived at the hospital at time 282517
Ambulance from event 992 has returned to station 14
Ambulance has arrived on the scene for event 998 at time 282647
Ambulance for event 997 has arrived at the hospital at time 282802
Ambulance for event 994 is returning to the station at 283022
Ambulance from event 995 has returned to station 5
Ambulance from event 990 has returned to station 44
Ambulance from event 991 has returned to station 19
Ambulance from event 994 has returned to station 17
Ambulance for event 998 has arrived at the hospital at time 283525
Ambulance for event 996 is returning to the station at 284003
Ambulance from event 996 has returned to station 17
Ambulance for event 997 is returning to the station at 284418
Ambulance from event 997 has returned to station 4
calling event id: 999 time: 285252 value: 97
Ambulance for event 998 is returni

(1000×11 DataFrame. Omitted printing of 6 columns
│ Row  │ id    │ dispatch_from │ waittime │ responsetime │ scenetime │
│      │ Int64 │ Int64         │ Float64  │ Float64      │ Float64   │
├──────┼───────┼───────────────┼──────────┼──────────────┼───────────┤
│ 1    │ 1     │ 1             │ 0.0      │ 5.0          │ 11.3833   │
│ 2    │ 2     │ 43            │ 0.0      │ 11.25        │ 13.35     │
│ 3    │ 3     │ 14            │ 0.0      │ 12.2333      │ 13.0      │
│ 4    │ 4     │ 34            │ 0.0      │ 10.1833      │ 12.2667   │
│ 5    │ 5     │ 17            │ 0.0      │ 9.76667      │ 11.25     │
│ 6    │ 6     │ 22            │ 0.0      │ 5.0          │ 16.25     │
│ 7    │ 7     │ 39            │ 0.0      │ 24.55        │ 12.1333   │
│ 8    │ 8     │ 8             │ 0.0      │ 5.0          │ 19.8333   │
│ 9    │ 9     │ 8             │ 0.0      │ 11.35        │ 12.75     │
│ 10   │ 10    │ 3             │ 0.0      │ 9.61667      │ 9.08333   │
⋮
│ 990  │ 990   │ 44      

In [8]:
"""batch call simulation"""
# batches = 12
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# using DelimitedFiles

# for i = 1:12
#     """ simulation parameters """
#     p = DeploymentProblem(
#               hourly_calls,
#               adjacent_nbhd,
#               coverage,
#               namb = namb,
#               train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#               )
#     test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#     #update 1: remove negative indices
#     start_ind = (i-1)*1000+1
#     end_ind = (i)*1000
#     println("running $namb ambuances & $start_ind to $end_ind calls")
#     test_calls_batch = test_calls[start_ind:end_ind,:]

#     # distribution of ambulances across all of the stations
#     x = amb_deployment[model_dict[model_name]][namb]

#     #establishes the dispatch problem
#     problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

#     #establishes the dispatch model
#     dispatch = ClosestDispatch(p, problem)

#     #@time df, guiArray = simulate_events!(problem, dispatch)
#     df, guiArray = simulate_events!(problem, dispatch)
#     print(mean(df["responsetime"]))
    
#     filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
#     open(filename, "w") do filename
#         writedlm(filename,  df["responsetime"])
#     end
#     print(filename)
# end

"batch call simulation"

In [22]:
print(mean(df["responsetime"]))

10.479283333333333

In [10]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

1000-element Array{Any,1}:
 gui_event("call responded", 83, 1, 1, 1, -1, 0, DateTime("2021-05-31T12:49:08.642"))
 gui_event("call responded", 38, 43, 2, 39, -1, 0, DateTime("2021-05-31T12:49:08.786"))
 gui_event("call responded", 129, 14, 3, 13, -1, 0, DateTime("2021-05-31T12:49:08.816"))
 gui_event("call responded", 107, 34, 4, 35, -1, 1, DateTime("2021-05-31T12:49:08.848"))
 gui_event("call responded", 113, 17, 5, 16, -1, 3, DateTime("2021-05-31T12:49:08.848"))
 gui_event("call responded", 51, 22, 6, 24, -1, 1, DateTime("2021-05-31T12:49:08.849"))
 gui_event("call responded", 76, 39, 7, 37, -1, 0, DateTime("2021-05-31T12:49:08.863"))
 gui_event("call responded", 67, 8, 8, 8, -1, 1, DateTime("2021-05-31T12:49:08.926"))
 gui_event("call responded", 52, 8, 9, 9, -1, 0, DateTime("2021-05-31T12:49:08.958"))
 gui_event("call responded", 67, 3, 10, 4, -1, 0, DateTime("2021-05-31T12:49:09.005"))
 gui_event("call responded", 67, 5, 11, 6, -1, 0, DateTime("2021-05-31T12:49:09.036"))
 gui_event

In [11]:
mean(df["responsetime"])

10.3443

In [23]:
using DelimitedFiles
filename = string(model_name,string(namb),"calls",string(ncalls),"closest_xuniform",".csv")
open(filename, "w") do filename
    writedlm(filename,  df["responsetime"])
end
print(filename)

Stochastic40calls1000closest_xuniform.csv

In [12]:
#using CSV
#filename = string(joshpath,"test/",model_name,string(namb),"calls",string(ncalls),".csv")
#CSV.write(filename, df, delim=",")